In [1]:
import pandas as pd

In [2]:
csv_file = "the-office-lines - scripts.csv"

In [3]:
OHCO = ['season', 'episode', 'scene', 'line', 'sentence', 'token']

In [4]:
raw = pd.read_csv(csv_file)

In [5]:
raw.rename(columns={'id': 'line'}, inplace=True)
cols = list(raw.columns)
cols.insert(3, cols.pop(cols.index('line')))
raw = raw[cols]

In [6]:
#raw.index.name = 'line'
raw

,season,episode,scene,line,line_text,speaker,deleted
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
1,1,1,1,2,"Oh, I told you. I couldn't close it. So...",Jim,False
2,1,1,1,3,So you've come to the master for guidance? Is ...,Michael,False
3,1,1,1,4,"Actually, you called me in here, but yeah.",Jim,False
4,1,1,1,5,"All right. Well, let me show you how it's done.",Michael,False
...,...,...,...,...,...,...,...
59904,9,23,112,59905,It all seems so very arbitrary. I applied for ...,Creed,False
59905,9,23,113,59906,I just feel lucky that I got a chance to share...,Meredith,False
59906,9,23,114,59907,I���m happy that this was all filmed so I can ...,Phyllis,False
59907,9,23,115,59908,I sold paper at this company for 12 years. My ...,Jim,False


In [7]:
sent_pat = r'[.?!;:]+'
SENTS = raw['line_text'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sentence')
    
cols_to_keep = ['season', 'episode', 'scene', 'line']  # Columns to keep
SENTS[cols_to_keep] = raw.loc[SENTS.index.get_level_values(0), cols_to_keep].values
SENTS.set_index(OHCO[:4], inplace=True)
SENTS

sentence
season episode scene line                                                    
1      1       1     1                                          All right Jim
                     1                        Your quarterlies look very good
                     1                          How are things at the library
                     1                                                       
                     2                                         Oh, I told you
...                                                                       ...
9      23      116   59909  I thought it was weird when you picked us to m...
                     59909   But all in all���I think an ordinary paper co...
                     59909       There���s a lot of beauty in ordinary things
                     59909                     Isn���t that kind of the point
                     59909                                                   

[173831 rows x 1 columns]

In [9]:
token_pat = r"[\s',-]+"
TOKENS = SENTS['sentence'].str.split(token_pat, expand=True).stack()\
    .to_frame('token')
    
TOKENS.index.names = OHCO[:5]

In [10]:
TOKENS

token
season episode scene line  sentence       
1      1       1     1     0           All
                           1         right
                           2           Jim
                           0              
                           1          Your
...                                    ...
9      23      116   59909 3          kind
                           4            of
                           5           the
                           6         point
                           0              

[826446 rows x 1 columns]

In [12]:
TOKENS['term_str'] = TOKENS.token.replace(r'[\W_]+', '', regex=True).str.lower()
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [13]:
VOCAB

,term_str,n
term_id,,
0,,115175
1,i,29075
2,you,24274
3,the,17719
4,to,16635
...,...,...
21227,hobos,1
21228,mim,1
21229,slats,1


In [14]:
def gather(ohco_level):
    global TOKENS
    level_name = OHCO[ohco_level-1].split('_')[0]
    df = TOKENS.groupby(OHCO[:ohco_level])\
        .token_str.apply(lambda x: x.str.cat(sep=' '))\
        .to_frame(f"{level_name}_str")
    return df

In [15]:
TOKENS.to_csv("office-TOKENS.csv")

In [16]:
VOCAB.to_csv("office-VOCAB.csv")